# QUDT

In [201]:
from pathlib import Path
import rdflib, ipywidgets as W, traitlets as T
from pyld import jsonld
import html
from urllib.request import urlretrieve
import IPython

In [94]:
QUDT_SPEC_URL = "http://qudt.org/2.0"

In [95]:
ONTO = Path("ontologies")
QUDT = ONTO / "QUDT"
QUDTS = QUDT / "schema"
QUDTV = QUDT / "vocab"
for qdir in [QUDTS, QUDTV]:
    if not qdir.exists():
        qdir.mkdir(parents=True)

In [147]:
NS = dict(initNs=dict(
    rdf=rdflib.RDF,
    rdfs=rdflib.RDFS,
    qudt=rdflib.URIRef("http://qudt.org/schema/qudt/")
))

In [96]:
for schema in ["SCHEMA_QUDT-v2.0.ttl", "SCHEMA_QUDT-SCIENCE-v2.0.ttl"]:
    if not (QUDTS / schema).exists():
        urlretrieve(f"{QUDT_SPEC_URL}/schema/{schema}", QUDTS / schema)

In [97]:
vocabs = [
    "DISCIPLINES",
    "UNITS-ACOUSTICS",
    "UNITS-ELECTROMAGNETISM",
    "UNITS-LIGHT",
    "UNITS-MECHANICS",
    "UNITS-PHYSICAL-CHEMISTRY-AND-MOLECULAR-PHYSICS",
    "UNITS-SPACE-AND-TIME",
    "UNITS-THERMODYNAMICS",
    "UNITS-BASE"
]
for vocab in vocabs:
    fname = f"VOCAB_QUDT-{vocab}-v2.0.ttl"
    print(fname)
    if not (QUDTV / fname).exists():
        urlretrieve(f"{QUDT_SPEC_URL}/vocab/{fname}", QUDTV / fname)

VOCAB_QUDT-DISCIPLINES-v2.0.ttl
VOCAB_QUDT-UNITS-ACOUSTICS-v2.0.ttl
VOCAB_QUDT-UNITS-ELECTROMAGNETISM-v2.0.ttl
VOCAB_QUDT-UNITS-LIGHT-v2.0.ttl
VOCAB_QUDT-UNITS-MECHANICS-v2.0.ttl
VOCAB_QUDT-UNITS-PHYSICAL-CHEMISTRY-AND-MOLECULAR-PHYSICS-v2.0.ttl
VOCAB_QUDT-UNITS-SPACE-AND-TIME-v2.0.ttl
VOCAB_QUDT-UNITS-THERMODYNAMICS-v2.0.ttl
VOCAB_QUDT-UNITS-BASE-v2.0.ttl


In [98]:
G = rdflib.Graph()
[G.parse(file=f.open(), format="turtle") for f in ONTO.rglob("*.ttl")];

In [166]:
unit_results = list(G.query("""
SELECT ?uri ?label ?abbrev ?description ?symbol
WHERE {
    ?uri rdf:type qudt:Unit .
    ?uri rdfs:label ?label .
    OPTIONAL {
        ?uri qudt:abbreviation ?abbrev .
        ?uri qudt:description ?description .
        ?uri qudt:symbol ?symbol .
    }
}
""", **NS))
len(unit_results)

683

In [192]:
class Unit(T.HasTraits):
    uri = T.CUnicode()
    label = T.CUnicode(allow_none=True)
    description = T.CUnicode(allow_none=True)
    abbreviation = T.CUnicode(allow_none=True)
    symbol = T.CUnicode(allow_none=True)
    
    def _repr_html_(self):
        min_uri = self.uri.split("/")[-1]
        symbol = self.symbol
        if not symbol:
            symbol = self.abbreviation or self.label or min_uri
        return f"""
        <label title="{html.escape(self.label or "")}: {html.escape(self.description or "")}">
            <i>{html.escape(symbol)}</i>
        </label>
        """

In [195]:
UNITS = {
    str(uri).split("/")[-1]: Unit(
        uri=uri, 
        description=description,
        abbreviation=abbreviation,
        symbol=symbol
    )
    for uri, label, abbreviation, description, symbol 
    in unit_results
}